In [5]:
import os
import sys
import random
import warnings

import numpy as np

import matplotlib.pyplot as plt

from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras.callbacks import EarlyStopping

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
K.set_session(tf.Session(config=config))

Using TensorFlow backend.


In [7]:
# Define IoU metric
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [8]:
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3

# Build U-Net model
inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.1) (c9)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[mean_iou])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 128, 128, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 16) 448         lambda_1[0][0]                   
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 128, 128, 16) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (

In [9]:
import h5py
f = h5py.File('imgandlabel.h5','r')

In [10]:
import scipy
from scipy import misc
i_height = 128
i_width = 128
X_resized = np.zeros([399,128,128,3])
Y_resized = np.zeros([399,128,128,1])
import cv2
for i in range(0,199):
    X_resized[2*i] = cv2.resize(f['img'][i].astype('uint8'), (i_height, i_width))
    X_resized[2*i+1] = cv2.flip(cv2.resize(f['img'][i].astype('uint8'), (i_height, i_width)),1)
    temp = cv2.resize(f['label'][i].astype('uint8'), (i_height, i_width), cv2.INTER_AREA )
    Y_resized[2*i] = temp.reshape(128,128,1)
    temp = cv2.flip(cv2.resize(f['label'][i].astype('uint8'), (i_height, i_width),cv2.INTER_AREA ),1)
    Y_resized[2*i+1] = temp.reshape(128,128,1)

X_resized = X_resized[0:397]
Y_resized = Y_resized[0:397]


## 여기까지 Data load

### 여기서 부터 Data Augmentation

In [11]:
import imgaug.augmenters as iaa
import imgaug as ia

seq = iaa.Sequential([
    iaa.Affine(rotate=(-45, 45)),  # rotate by -45 to 45 degrees (affects segmaps)
    iaa.GammaContrast((0.5, 2.0), per_channel=True)
], random_order=True)

segmaps_aug=np.zeros([397,128,128,1])
images_aug=np.zeros([397,128,128,3])
segmaps_aug_i = []
images_aug_i = []
for i in range(0,397):
    image = X_resized[i].astype('uint8')
    segmap = ia.SegmentationMapOnImage(Y_resized[i], shape = (128,128,1))
    
    images_aug_i, segmaps_aug_i = seq(image=image, segmentation_maps=segmap)
    segmaps_aug[i] = np.array(segmaps_aug_i.arr)
    images_aug[i] = images_aug_i

X_resized = X_resized/255
Y_resized = Y_resized/255
images_aug = images_aug/255

## Data Arrange

In [12]:
X_resized = np.concatenate((X_resized,images_aug))
Y_resized = np.concatenate((Y_resized,segmaps_aug)) 

In [13]:
# import scipy
# from scipy import misc
# i_height = 128
# i_width = 128
# X_resized = np.zeros([199,128,128,3])
# Y_resized = np.zeros([199,128,128,1])
# for i in range(0,199):
#     X_resized[i] = scipy.misc.imresize(f['img'][i].astype('uint8'), (i_height, i_width,3))
#     temp = scipy.misc.imresize(f['label'][i].astype('uint8'), (i_height, i_width))
#     Y_resized[i] = temp.reshape(128,128,1)

In [14]:
from sklearn.model_selection import train_test_split
# Data Split
X_train, X_test, y_train, y_test = train_test_split(X_resized, Y_resized, test_size=0.3)

In [15]:
early_stopping = EarlyStopping()
results = model.fit(X_train, y_train, batch_size=16, epochs=50,validation_data=(X_test, y_test))

Train on 555 samples, validate on 239 samples
Epoch 1/50
555/555 [==============================] - 14s 25ms/step - loss: 0.2727 - mean_iou: 0.4601 - val_loss: 0.1591 - val_mean_iou: 0.4801
Epoch 2/50
555/555 [==============================] - 3s 6ms/step - loss: 0.1571 - mean_iou: 0.4809 - val_loss: 0.1490 - val_mean_iou: 0.4811
Epoch 3/50
555/555 [==============================] - 3s 5ms/step - loss: 0.1499 - mean_iou: 0.4810 - val_loss: 0.1494 - val_mean_iou: 0.4811
Epoch 4/50
555/555 [==============================] - 3s 5ms/step - loss: 0.1486 - mean_iou: 0.4812 - val_loss: 0.1492 - val_mean_iou: 0.4811
Epoch 5/50
555/555 [==============================] - 3s 5ms/step - loss: 0.1476 - mean_iou: 0.4811 - val_loss: 0.1575 - val_mean_iou: 0.4811
Epoch 6/50
555/555 [==============================] - 3s 5ms/step - loss: 0.1470 - mean_iou: 0.4811 - val_loss: 0.1445 - val_mean_iou: 0.4811
Epoch 7/50
555/555 [==============================] - 3s 5ms/step - loss: 0.1429 - mean_iou: 0.4811 

In [ ]:
model.save("UNET.h5")

In [ ]:
print(y_train.shape)

In [ ]:
print(X_test.shape)

In [ ]:
from matplotlib import gridspec
%matplotlib inline

nrow = 159
ncol = 2
fig = plt.figure(figsize=(ncol+1, nrow+1))
Y = model.predict(X_test)

gs = gridspec.GridSpec(nrow, ncol,
         wspace=0.0, hspace=0.0, 
         top=1.-0.5/(nrow+1), bottom=0.5/(nrow+1), 
         left=0.5/(ncol+1), right=1-0.5/(ncol+1)) 

for i in range(0,nrow):
    Y_out = Y[i,:,:,:].reshape(128,128)
    ax= plt.subplot(gs[i,0])
    ax.imshow(X_test[i,:,:,:], interpolation='nearest', aspect='auto')
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax= plt.subplot(gs[i,1])
    ax.imshow(X_test[i,:,:,:], interpolation='nearest', aspect='auto')
    ax.imshow(Y_out,alpha=0.5, interpolation='nearest', aspect='auto')
    ax.set_xticklabels([])
    ax.set_yticklabels([])

In [ ]:
from matplotlib import gridspec

%matplotlib inline
nrow = 159
ncol = 2
fig = plt.figure(figsize=(ncol+1, nrow+1))
Y = model.predict(X_train)

gs = gridspec.GridSpec(nrow, ncol,
         wspace=0.0, hspace=0.0, 
         top=1.-0.5/(nrow+1), bottom=0.5/(nrow+1), 
         left=0.5/(ncol+1), right=1-0.5/(ncol+1)) 

for i in range(0,159):
    Y_out = Y[i,:,:,:].reshape(128,128)
    ax= plt.subplot(gs[i,0])
    ax.imshow(X_train[i,:,:,:], interpolation='nearest', aspect='auto')
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax= plt.subplot(gs[i,1])
    ax.imshow(X_train[i,:,:,:], interpolation='nearest', aspect='auto')
    ax.imshow(Y_out,alpha=0.5, interpolation='nearest', aspect='auto')
    ax.set_xticklabels([])
    ax.set_yticklabels([])

In [ ]:
i_height = 1080
i_width = 1920
X_recon = np.zeros([nrow,1080,1920,3])
Y_recon = np.zeros([nrow,1080,1920])
for i in range(0,nrow):
    X_recon[i] = scipy.misc.imresize(X_test[i], (i_height, i_width,3))
    Y_recon[i] = scipy.misc.imresize(Y[i].reshape(128,128), (i_height, i_width))

In [ ]:
print(Y.shape)

In [ ]:
plt.imshow(X_recon[0].astype('uint8'))
plt.imshow(Y_recon[0])

In [ ]:
from matplotlib import gridspec
nrow = 159
ncol = 2
fig = plt.figure(figsize=(ncol+1, nrow+1))

gs = gridspec.GridSpec(nrow, ncol,
         wspace=0.0, hspace=0.0, 
         top=1.-0.5/(nrow+1), bottom=0.5/(nrow+1), 
         left=0.5/(ncol+1), right=1-0.5/(ncol+1)) 

for i in range(0,nrow):
    ax= plt.subplot(gs[i,0])
    ax.imshow(X_recon[i].astype('uint8'), interpolation='nearest', aspect='auto')
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax= plt.subplot(gs[i,1])
    ax.imshow(X_recon[i].astype('uint8'), interpolation='nearest', aspect='auto')
    ax.imshow(Y_recon[i],alpha=0.5, interpolation='nearest', aspect='auto')
    ax.set_xticklabels([])
    ax.set_yticklabels([])

In [ ]:
import scipy.io

In [ ]:
scipy.io.savemat('segmentation_data.mat', mdict={'img': X_test,'label' : Y}, format='5') 

In [ ]:
import cv2
cv2.cornerHarris(gray,2,3,0.04)
cv2.imshow

In [ ]:
model = load_model('UNET.h5')

In [ ]:
%matplotlib qt

In [ ]:
plt.imshow(Y[0].reshape(128,128))
plt.show()

In [ ]:
videoFile1 = 'video1.mp4'
cap = cv2.VideoCapture(videoFile1)
while(cap.isOpened()):
    ret, frame = cap.read()
    
    if ret:
        cv2.imshow('video',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
        
        
        
cap.release()
cv2.destroyAllWindows()
        
        
        

In [ ]:
videoFile1 = 'video1.mp4'
cap = cv2.VideoCapture(videoFile1)
while(cap.isOpened()):
    ret, frame = cap.read()
    
    if ret:
        cv2.imshow('video',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
        
        
        
cap.release()
cv2.destroyAllWindows()

In [75]:
import cv2
import numpy as np
videoFile1 = 'VideoForObjectDetection.mp4'

cap = cv2.VideoCapture(videoFile1)
frameCount = 500
frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

buf = np.empty((frameCount, frameHeight, frameWidth, 3))
fc = 0
cnt = 0;
while(fc<frameCount):
    ret, img = cap.read()
    if cnt%10 == 0:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        buf[fc] = img_rgb
        fc += 1
    cnt += 1
        
cap.release()

error: OpenCV(4.1.2) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [76]:
import scipy
from scipy import misc
i_height = 128
i_width = 128
X_other = np.zeros([frameCount,128,128,3])
import cv2
for i in range(0,frameCount):
    X_other[i] = cv2.resize(buf[i], (i_height, i_width))

In [77]:
Y = model.predict(X_other/255)

In [ ]:
imgnum = 300
plt.subplot(2,1,1)
plt.imshow(X_other[imgnum].astype('uint8'))
plt.subplot(2,1,2)
plt.imshow(Y[imgnum].reshape(128,128))

In [ ]:
print(X_resized[0].shape)

In [ ]:
import skvideo.io 
cap = VideoCapture(filename)
cap.open()
retval, image = cap.read()


In [78]:
Y_reshape = Y.reshape(frameCount,128,128)
scipy.io.savemat('post_data3.mat', mdict={'img': X_other,'label' : Y}, format='5') 